In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
import sys
import cv2
from scipy.misc import  imread,imresize
import operator

print("Packages loaded")

# Load datasets from npz file
cwd = os.getcwd()
loadpath = cwd + "/data/custom_data.npz"
l = np.load(loadpath)

# See what's inside of the npz file
print(l.files)

# Parse data
trainimg = l['trainimg']
trainlabel = l['trainlabel']
testimg = l['testimg']
testlabel = l['testlabel']
imgsize = l['imgsize']
use_gray = l['use_gray']

# ntrain contains the number of images in the trainingset (trainimg.shape ->[553 x 4096])
ntrain = trainimg.shape[0]
# dim contains the number of pixels of an image
dim = trainimg.shape[1]
# ntest contains the number of images in the testset (testimg.shape ->[370 x 4096])
ntest = testimg.shape[0]
# nclass contains the number of labels (trainlabel.shape ->[553 x 2])
nclass = trainlabel.shape[1] # 

print("%d train images loaded" % (ntrain))
print("%d test images loaded" % (ntest))
print("%d dimensional input" % (dim))
print("Image size is %s" % (imgsize))
print("%d classes" % (nclass))

Packages loaded
['testimg', 'trainimg', 'use_gray', 'trainlabel', 'imgsize', 'testlabel']
752 train images loaded
502 test images loaded
4096 dimensional input
Image size is [64 64]
4 classes


In [2]:
#define variables
tf.set_random_seed(0)
n_input = dim
n_output = nclass
if use_gray:
    weights = {
        'wc1': tf.Variable(tf.random_normal([5, 5, 1, 128], stddev=0.1), name="wc1"),
        'wc2': tf.Variable(tf.random_normal([5, 5, 128, 128], stddev=0.1), name="wc2"),
        'wd1': tf.Variable(tf.random_normal(
            [(int)(imgsize[0]/4*imgsize[1]/4)*128, 128], stddev=0.1), name="wd1"),
        'wd2': tf.Variable(tf.random_normal([128, n_output], stddev=0.1), name="wd2")
    }
else:
    weights = {
        'wc1': tf.Variable(tf.random_normal([5, 5, 3, 128], stddev=0.1), name="wc1"),
        'wc2': tf.Variable(tf.random_normal([5, 5, 128, 128], stddev=0.1), name="wc2"),
        'wd1': tf.Variable(tf.random_normal(
            [(int)(imgsize[0] / 4 * imgsize[1] / 4) * 128, 128], stddev=0.1), name="wd1"),
        'wd2': tf.Variable(tf.random_normal([128, n_output], stddev=0.1), name="wd2")
    }
biases = {
    'bc1': tf.Variable(tf.random_normal([128], stddev=0.1), name="bc1"),
    'bc2': tf.Variable(tf.random_normal([128], stddev=0.1), name="bc2"),
    'bd1': tf.Variable(tf.random_normal([128], stddev=0.1), name="bd1"),
    'bd2': tf.Variable(tf.random_normal([n_output], stddev=0.1), name="bd2")
}

#define network
def conv_basic(_input, _w, _b, _keepratio, _use_gray):
    # INPUT
    if _use_gray:
        _input_r = tf.reshape(_input, shape=[-1, imgsize[0], imgsize[1], 1])
    else:
        _input_r = tf.reshape(_input, shape=[-1, imgsize[0], imgsize[1], 3])
    # CONVOLUTION LAYER 1
    _conv1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(_input_r
      , _w['wc1'], strides=[1, 1, 1, 1], padding='SAME'), _b['bc1']))
    _pool1 = tf.nn.max_pool(_conv1, ksize=[1, 2, 2, 1]
                            , strides=[1, 2, 2, 1], padding='SAME')
    _pool_dr1 = tf.nn.dropout(_pool1, _keepratio)
    # CONVOLUTION LAYER 2
    _conv2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(_pool_dr1
                                                    , _w['wc2'], strides=[1, 1, 1, 1], padding='SAME'), _b['bc2']))
    _pool2 = tf.nn.max_pool(_conv2, ksize=[1, 2, 2, 1]
                            , strides=[1, 2, 2, 1], padding='SAME')
    _pool_dr2 = tf.nn.dropout(_pool2, _keepratio)
    # VECTORIZE
    _dense1 = tf.reshape(_pool_dr2
                         , [-1, _w['wd1'].get_shape().as_list()[0]])
    # FULLY CONNECTED LAYER 1
    _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1, _w['wd1']), _b['bd1']))
    _fc_dr1 = tf.nn.dropout(_fc1, _keepratio)
    # FULLY CONNECTED LAYER 2
    _out = tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2'])
    # RETURN
    out = {
        'out': _out
    }
    return out
print ("NETWORK READY")

#define functions
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])
keepratio = tf.placeholder(tf.float32)

# Functions! 
_pred = conv_basic(x, weights, biases, keepratio, use_gray)['out']
_corr = tf.equal(tf.argmax(_pred,1), tf.argmax(y,1)) # Count corrects
accr = tf.reduce_mean(tf.cast(_corr, tf.float32)) # Accuracy
print ("FUNCTIONS READY")



init = tf.initialize_all_variables()

# Launch the graph
sess = tf.Session()
sess.run(init)
print("start restoring.")


#Load weights from saver 
saver = tf.train.Saver(max_to_keep=3) 
saver.restore(sess, "C:/Users/nur20/Documents/GitHub/TensorFlow-Master/save/custom_basic_cnn_idle.ckpt")
print("Model restored.")

imageName="C:/Users/nur20/Documents/GitHub/TensorFlow-Master/img_dataset/George_W_Bush/George_W_Bush_0001.jpg"

#imageName="/home/soeren/data/Facerecognition/img_dataset/Arnold_Schwarzenegger/Arnold_Schwarzenegger_0040.jpg"
#imageName="/home/soeren/data/Facerecognition/img_dataset/Vladimir_Putin/Vladimir_Putin_0032.jpg"
#imageName="/home/soeren/data/Facerecognition/img_dataset/Junichiro_Koizumi/Junichiro_Koizumi_0014.jpg"
#imageName="/home/soeren/data/Facerecognition/img_dataset/George_W_Bush/George_W_Bush_0031.jpg"
#imageName="/home/soeren/data/Facerecognition/img_dataset/Soeren_Erichsen/6.png"
#imageName="/home/soeren/data/Facerecognition/img_dataset/Pascal/7.png"
#imageName="/home/soeren/data/Facerecognition/img_dataset/Apple/7.png"


img_gray  = cv2.imread(imageName,0) #load grayscale
#img_gray= rgb2gray(imread(imageName))
img_gray_resize=imresize(img_gray, [64, 64])/255. #resize image to [64x64]
img_grayvec   = np.reshape(img_gray_resize, (1, -1)) #reshape matrix to vector
#cv2.imshow("bla",img_gray_resize)
#cv2.waitKey(1000)
#print img_grayvec
print (img_grayvec.shape)


predictiton=sess.run(tf.nn.softmax(_pred), feed_dict={x: img_grayvec,keepratio:1.}) #make prediction
print (predictiton)

index, value = max(enumerate(predictiton[0]), key=operator.itemgetter(1)) #find highest value in output vector

className=""
if index==0:
  className="al gore"
elif index ==1:
  className="Bush"
elif index ==2:
  className="tyler"
elif index ==3:
  className="jobs"


print ("Prediciton is class '%s' with accuracy %0.3f"%(className,value))

# Do some work with the model
test_acc = sess.run(accr, feed_dict={x: testimg, y: testlabel, keepratio:1.})
    
print (" Test accuracy: %.3f" % (test_acc))


sess.close()
print ("Session closed.")

NETWORK READY
FUNCTIONS READY
Instructions for updating:
Use `tf.global_variables_initializer` instead.
start restoring.
INFO:tensorflow:Restoring parameters from C:/Users/nur20/Documents/GitHub/TensorFlow-Master/save/custom_basic_cnn_idle.ckpt
Model restored.
(1, 4096)
[[  9.89172339e-01   6.70049167e-06   1.06630739e-06   1.08198598e-02]]
Prediciton is class 'al gore' with accuracy 0.989
 Test accuracy: 0.131
Session closed.
